# Metacritic All Time Games Statistics (1995 - 2018)

For this project, we used a dataset on Metacritic Games Reviews and Ratings from this kaggle page: https://www.kaggle.com/skateddu/metacritic-all-time-games-stats. Our goal is to ....


First, we import the necessary libraries:

In [1]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import numpy as np
import requests
import json
import requests
import pymongo

Next we import the dataset and preview the dataframe in pandas.

In [2]:
# Import dataset csv and preview
df = pd.read_csv('metacritic_games.csv')
df.head()

,name,platform,developer,publisher,genre,players,rating,attribute,release_date,link,critic_positive,critic_neutral,critic_negative,metascore,user_positive,user_neutral,user_negative,user_score
0,Command & Conquer,PC,Westwood Studios,Virgin Interactive,Sci-Fi,01-Apr,T,NaN,"Aug 31, 1995",/game/pc/command-conquer,5,0,0,94,47,0,1,8.9
1,Full Throttle,PC,LucasArts,LucasArts,Adventure,NaN,NaN,NaN,"Apr 30, 1995",/game/pc/full-throttle,6,2,0,86,18,1,0,8.7
2,Battle Arena Toshinden,PS,Tamsoft,SCEA,Action,01-Feb,T,NaN,"Sep 9, 1995",/game/playstation/battle-arena-toshinden,1,3,0,69,1,0,1,5.8
3,Sid Meier's Civilization II,PC,MPS Labs,MicroProse,Strategy,1 Player,K-A,NaN,"Feb 29, 1996",/game/pc/sid-meiers-civilization-ii,7,0,0,94,46,0,1,8.9
4,Quake,PC,id Software,id Software,Action,Jan-16,M,NaN,"Jun 22, 1996",/game/pc/quake,9,0,0,94,84,4,1,8.8


## Data Cleaning

### Removing incomplete rows and redundant columns

We begin data cleaning by dropping rows with missing values in developer and publisher.

In [3]:
# Drop rows with missing values in developer and publisher
df.dropna(subset=['developer', 'publisher'], inplace = True)
df.reset_index(inplace = True)
df = df.drop(['index'], axis = 1)

The following columns are dropped as they are not relevant to our research question:

In [4]:
# Drop redundant columns
df = df.drop(['players', 'attribute', 'link', 'critic_positive',
              'critic_neutral', 'critic_negative','user_positive',
              'user_neutral', 'user_negative'], axis = 1)

### Obtaining Release Year

Next, we sliced the 'release_date' to obtain the year of release and replaced the column with the appropriate 'release_year' data.

In [5]:
# Change Release Date to Release Year
year_released = []
for i in range(0,len(df['release_date'])):    
    year_released.append(df.release_date[i].split()[2])
    
df['release_date'] = year_released
df.rename(columns={"release_date": "release_year"}, inplace = True)

### Rating

Rating columns are adjusted by replacing the letters with the actual rating description, as follows:
* **E** = Everyone
* **EC** = Early Childhood 3+
* **E10+** = Everyone 10+
* **T** = Teen 13+
* **M** = Mature 17+
* **AO** = Adults Only 18+
* **RP** or **K-A** = Rating Pending
* **Null values** are replaced as Rating Pending

In [6]:
# Add Description to Rating column and remove null values
for rating in df['rating']:
    if rating == 'E':
        df.rating.replace(to_replace = rating, value = 'Everyone', inplace=True)
    elif rating == 'EC':
        df.rating.replace(to_replace = rating, value = 'Early Childhood 3+', inplace=True)
    elif rating == 'E10+':
        df.rating.replace(to_replace = rating, value = 'Everyone 10+', inplace=True)
    elif rating == 'T':
        df.rating.replace(to_replace = rating, value = 'Teen 13+', inplace=True)
    elif rating == 'M':
        df.rating.replace(to_replace = rating, value = 'Mature 17+', inplace=True)
    elif rating == 'AO':
        df.rating.replace(to_replace = rating, value = 'Adults Only 18+', inplace=True)
    elif rating == 'RP' or rating == 'K-A' or type(rating) == float:
        df.rating.replace(to_replace = rating, value = 'Rating Pending', inplace=True)

### Adjusting User Score

The user_score column contains 0 null values, however 'tbd' is present, representing the user score is to be determined. As there are 2000 + values of 'tbd', we will replace the values with random-assigned values with weighted probabilities from other user review scores

In [7]:
# Assign 'tbd' values with random-assigned values with weighted probablities
choice = sorted(df[df.user_score != 'tbd']['user_score'].unique())
prob = df[df.user_score != 'tbd']['user_score'].value_counts(normalize=True).sort_index()
df['user_score'] = df.user_score.map(lambda x: np.random.choice(choice, 1, p= prob)[0] if x == 'tbd' else x)

For comparison purposes, we will adjust the user_score to be in the same format as the metascore by multiplying all values by 10 as user_score is rated on a scale of 10 while metascore is rated on a scale of 100. Appropriate datatype conversion will be performed to the values.

In [8]:
# Change datatype from str to float
df[['user_score']] = df[['user_score']].astype(float)

# Change user_score from a scale of 10 to a scale of 100 by multiplying all values by 10
df['user_score'] = df['user_score'].apply(lambda x: x * 10 )

# Change datatype from float to int
df[['user_score']] = df[['user_score']].astype(int)

In [9]:
print(df.shape)
df.head()

(20380, 9)


,name,platform,developer,publisher,genre,rating,release_year,metascore,user_score
0,Command & Conquer,PC,Westwood Studios,Virgin Interactive,Sci-Fi,Teen 13+,1995,94,89
1,Full Throttle,PC,LucasArts,LucasArts,Adventure,Rating Pending,1995,86,87
2,Battle Arena Toshinden,PS,Tamsoft,SCEA,Action,Teen 13+,1995,69,58
3,Sid Meier's Civilization II,PC,MPS Labs,MicroProse,Strategy,Rating Pending,1996,94,89
4,Quake,PC,id Software,id Software,Action,Mature 17+,1996,94,88


Great! We now have a clean dataframe in the right format with no missing values. The dataframe has a shape of **9 columns x 20380 rows**. We can now begin our data analysis and hypothesis testing.

## Data Analysis 